In [1]:
# Import dependencies
import json
import requests
import pandas as pd
from flask import Flask, redirect, request, url_for, render_template, session
from sleeperpy import User, Leagues
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [2]:
#def franchise_comparison():
# user_id = session.get("user_id")
# league_id = session.get("league_id")
user_id = "725907041763303424"
league_id = "1049754172834455552"

In [4]:
backendapi_franchisecomparison_url = f'https://ygn5at2pt5zgycacip4t6kvrey0ayfrl.lambda-url.us-east-2.on.aws//franchise_comparison/{user_id}/{league_id}'
response = requests.get(backendapi_franchisecomparison_url)
response

<Response [200]>

In [8]:
response.encoding

'utf-8'

In [ ]:
# Parse the data
backend_response = pd.DataFrame(response)


In [ ]:
# Set datatypes
backend_response['id_sleeper'] = backend_response['id_sleeper'].astype(str)
backend_response['team_name'] = backend_response['team_name'].astype(str)

# Aggregate the predicted points
player_sums = backend_response.groupby(['id_sleeper'])['predicted_fantasy_points'].sum().to_frame().reset_index()
total_franchise_values = backend_response.groupby('team_name')['predicted_fantasy_points'].sum().round(0).to_frame().reset_index()
# Merge back in the player info
player_info = backend_response.drop(columns=['predicted_fantasy_points', 'week_of_season']).drop_duplicates(ignore_index=True)
franchise_comparisons = player_sums.merge(player_info, on='id_sleeper', how='left')
# Visualize the data
fig = px.bar(franchise_comparisons, 
            x="team_name", 
            y='predicted_fantasy_points', 
            color="position", 
            text='full_name', 
            color_discrete_map={ 
                "QB": "#9CCFD6", 
                "RB": "#DEB2AF", 
                "WR": "#97BFF0", 
                "TE": "#CEE1B2", 
                "K": "#F0C397", 
                "DEF": "#F0DB97"}, 
            category_orders={
                "position": ["QB", "RB", "WR", "TE", "K", "DEF"]},
            hover_name="full_name",
            hover_data={
                'predicted_fantasy_points':True,
                'full_name':False, 'position':False, 'team_name':False
                },
            labels={
                "team_name":"Franchise",
                'predicted_fantasy_points':"Predicted Points",
            }
            )
fig.update_traces(
    textposition='inside',
)
fig.update_yaxes(
    visible=False
)
fig.add_trace(go.Scatter(
            x=total_franchise_values['team_name'], 
            y=total_franchise_values['predicted_fantasy_points'],
            text=total_franchise_values['predicted_fantasy_points'],
            mode='text',
            textposition='top center',
            textfont=dict(
                size=12,
            ),
            showlegend=False
        ))
fig.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="Franchise Comparison",
            font_family="Input Serif",
            showlegend=False,
            xaxis_title=None
            )
# graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
# return render_template('compareFranchises.html', graphJSON=graphJSON)

In [ ]:
fig.show()